# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame


In [2]:
#Loading the csv file
weather_data = '../WeatherPy/City_Weather_Data1.csv'
weather = pd.read_csv(weather_data)
weather = weather.drop(columns=['Unnamed: 0'])
weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,thompson,55.7435,-97.8558,-27.40,68,40,12.66,CA,1612662454
1,luderitz,-26.6481,15.1594,67.42,70,0,22.70,NaN,1612662656
2,kapaa,22.0752,-159.3190,73.99,49,40,6.91,US,1612662240
3,kousseri,12.0769,15.0306,65.98,23,41,9.98,CM,1612662937
4,punta arenas,-53.1500,-70.9167,51.80,66,0,17.27,CL,1612662441


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)
weather = weather.dropna()

In [4]:
#Adding a heat map
location = weather[['Lat','Lng']].astype(float)
humidity1 = weather['Humidity'].astype(float)

fig1 = gmaps.figure(center = [0,0],zoom_level = 2)
heat = gmaps.heatmap_layer(location,weights=humidity1,dissipating = False, max_intensity = 100,point_radius = 2)

fig1.add_layer(heat)
fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

 Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [5]:
#Creating the same data frame as weather
weather_df = weather
weather_df = weather_df.loc[(weather_df['Max Temp'] < 80 ) & (weather_df['Max Temp'] > 70) & (weather_df['Wind Speed']<10) 
                           & (weather_df['Cloudiness']== 0)]

weather_df = weather_df.dropna()
weather_df.tail()
print('The number of cities that meet all 3 conditions')
print(weather_df.count())
print('--------------------------')
print('The number of cities that did not meet all 3 conditions')
print(weather.count())

The number of cities that meet all 3 conditions
City          22
Lat           22
Lng           22
Max Temp      22
Humidity      22
Cloudiness    22
Wind Speed    22
Country       22
Date          22
dtype: int64
--------------------------
The number of cities that did not meet all 3 conditions
City          551
Lat           551
Lng           551
Max Temp      551
Humidity      551
Cloudiness    551
Wind Speed    551
Country       551
Date          551
dtype: int64


In [6]:
### Hotel Map
#* Store into variable named `hotel_df`.
#* Add a "Hotel Name" column to the DataFrame.
#* Set parameters to search for hotels with 5000 meters.
#* Hit the Google Places API for each city's coordinates.
#* Plot markers on top of the heatmap.

In [7]:
#We only want 10 pins in the next heatmap
hotel_df_ten =weather_df.head(10)

hotel_df_ten


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,pathein,16.7833,94.7333,78.94,63,0,8.16,MM,1612662938
40,villa carlos paz,-31.4241,-64.4978,75.00,57,0,5.99,AR,1612662940
52,te anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612662649
72,chontalpa,17.7000,-93.5167,71.91,81,0,5.68,MX,1612662944
82,lazaro cardenas,17.9583,-102.2000,73.90,75,0,1.59,MX,1612662507
100,vao,-22.6667,167.4833,78.03,80,0,5.46,NC,1612662491
109,mahebourg,-20.4081,57.7000,75.00,86,0,3.00,MU,1612662438
112,santa fe,-31.6333,-60.7000,75.00,65,0,6.40,AR,1612662765
231,benemerito de las americas,16.5167,-90.6439,72.57,81,0,1.83,MX,1612662959
270,corrientes,-27.4806,-58.8341,73.00,73,0,4.99,AR,1612662964


In [8]:
#Setting up a new column to store hotel name and get the url set up
hotel_df = hotel_df_ten.copy()
hotel_df['Hotel Name'] = ' '

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'radius':5000,'types':'lodging','key':g_key
}

In [9]:
#Getting the hotel name info and storing it in the dataframe
for index, row in hotel_df.iterrows():
    lat_h = row['Lat']
    lng_h = row['Lng']
    params['location'] = f'{lat_h},{lng_h}'
    response1 = requests.get(base_url, params = params).json()
    
    try:
        hotel_df.loc[index,'Hotel Name'] = response1['results'][0]['name']
    except:
        hotel_df.loc[index,'Hotel Name'] = 'NA-not within radius'
        

In [10]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,pathein,16.7833,94.7333,78.94,63,0,8.16,MM,1612662938,Pathein Hotel
40,villa carlos paz,-31.4241,-64.4978,75.00,57,0,5.99,AR,1612662940,Monte Carlo
52,te anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612662649,Kingsgate Hotel Te Anau
72,chontalpa,17.7000,-93.5167,71.91,81,0,5.68,MX,1612662944,NA-not within radius
82,lazaro cardenas,17.9583,-102.2000,73.90,75,0,1.59,MX,1612662507,Collection O Hotel Quinta Antigua
100,vao,-22.6667,167.4833,78.03,80,0,5.46,NC,1612662491,Hotel Kou-Bugny
109,mahebourg,-20.4081,57.7000,75.00,86,0,3.00,MU,1612662438,Shandrani Beachcomber Resort & Spa
112,santa fe,-31.6333,-60.7000,75.00,65,0,6.40,AR,1612662765,Conquistador Hotel
231,benemerito de las americas,16.5167,-90.6439,72.57,81,0,1.83,MX,1612662959,"Tres Lagunas, Selva Lacandona"
270,corrientes,-27.4806,-58.8341,73.00,73,0,4.99,AR,1612662964,Gran Hotel Guaraní


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
fig_h = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig1.add_layer(markers)
fig1



Figure(layout=FigureLayout(height='420px'))